# APVC – Exercícios – Espaços de cor, Binarização

In [161]:
### imports

# ! pip install opencv-python
import cv2
import numpy as np

## Exercício 1 (acesso aos pixels, espaços de cor)

Desenvolva um script em Python que lê uma imagem e calcula o seu “negativo” na parte
da imagem correspondente a um quadrado (ou retângulo) centrado. As dimensões do
quadrado (ou retângulo) correspondem a metade das resoluções da imagem original.

In [162]:
img = cv2.imread("images/lenna.png")
img.shape

(512, 512, 3)

In [163]:
# Definir tamanho do recorte
n = 150

# Clonar a imagem original para modificar
imgMasked = img.copy()

# Coordenadas do centro
x_center, y_center = img.shape[1] // 2, img.shape[0] // 2

# Recortar a região central
imgCrop = img[y_center - n:y_center + n, x_center - n:x_center + n]

# Inverter as cores do recorte
imgCrop = cv2.bitwise_not(imgCrop)

# Substituir a região original pela versão invertida
imgMasked[y_center - n:y_center + n, x_center - n:x_center + n] = imgCrop

# Mostrar as imagens
cv2.imshow("Masked Image", imgMasked)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Exercício 2 (acesso aos pixels, conversões entre espaços de cor)

Pretende-se construir uma imagem que mostra a evolução da tonalidade das cores à
medida que se varia a componente Hue, mantendo a Saturation e Value com valores
constantes (255).

In [164]:
altura = 500
largura = 1080

hue = np.tile(np.linspace(0, 179, largura, dtype=np.uint8), (altura, 1))
hue

array([[  0,   0,   0, ..., 178, 178, 179],
       [  0,   0,   0, ..., 178, 178, 179],
       [  0,   0,   0, ..., 178, 178, 179],
       ...,
       [  0,   0,   0, ..., 178, 178, 179],
       [  0,   0,   0, ..., 178, 178, 179],
       [  0,   0,   0, ..., 178, 178, 179]], dtype=uint8)

In [165]:
# Criar matriz para os canais Saturação (S) e Valor (V), fixos em 255
saturation = np.full((altura, largura), 255, dtype=np.uint8)
value = np.full((altura, largura), 255, dtype=np.uint8)

# Juntar os canais H, S e V
hsv_image = cv2.merge([hue, saturation, value])
print(hsv_image)

# Converter para BGR
bgr_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

[[[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 ...

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]

 [[  0 255 255]
  [  0 255 255]
  [  0 255 255]
  ...
  [178 255 255]
  [178 255 255]
  [179 255 255]]]


In [166]:
# Mostrar a imagem
cv2.imshow("Hue Gradient", bgr_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Exercício 3 (espaços de cor, binarização e máscaras)

Utilize máscaras binárias de forma a conseguir que, a partir de uma imagem com múltiplos objetos coloridos, se consiga chegar a uma imagem onde apareçam os objetos de uma determinada cor, ficando o resto da imagem a preto.

In [173]:
def isColorHue(value, color="blue"):
    if not (0 <= value <= 255):
        raise ValueError("O valor deve estar no intervalo [0, 255].")

    color_ranges = {
        "red": (0, 10) or (170, 255),  # Tons de vermelho estão nos extremos do círculo de matizes
        "orange": (10, 25),
        "yellow": (25, 45),
        "green": (45, 85),
        "cyan": (85, 105),
        "blue": (105, 135),
        "magenta": (135, 165),
        "pink": (165, 185),
        "purple": (185, 200)
    }

    if color not in color_ranges:
        raise ValueError(f"Cor inválida: {color}. Escolha entre {', '.join(color_ranges.keys())}.")

    min_val, max_val = color_ranges[color]

    # Caso especial para o vermelho, que está nos extremos do espectro circular
    if color == "red":
        return (0 <= value <= 10) or (170 <= value <= 255)
    else:
        return min_val <= value <= max_val

In [174]:
img = cv2.imread("images/legos.jpg")
img.shape

hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

treshold = 150
for i in hsv_img:
    for j in i:
        if not isColorHue(j[0], "orange"):
            j[2] = 0
        if j[1] < treshold:
            j[2] = 0
hsv_img

array([[[177, 252,   0],
        [177, 254,   0],
        [177, 255,   0],
        ...,
        [ 26, 243,   0],
        [ 26, 227,   0],
        [ 26, 232,   0]],

       [[177, 252,   0],
        [177, 254,   0],
        [177, 255,   0],
        ...,
        [ 26, 225,   0],
        [ 26, 228,   0],
        [ 26, 233,   0]],

       [[177, 251,   0],
        [177, 254,   0],
        [177, 255,   0],
        ...,
        [ 26, 218,   0],
        [ 26, 229,   0],
        [ 26, 234,   0]],

       ...,

       [[178, 217,   0],
        [179, 197,   0],
        [  2, 219,   0],
        ...,
        [179, 253,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[178, 208,   0],
        [179, 214,   0],
        [  3, 227,   0],
        ...,
        [179, 253,   0],
        [  0, 252,   0],
        [  1, 255,   0]],

       [[179, 218,   0],
        [  0, 237,   0],
        [  3, 220,   0],
        ...,
        [  0, 250,   0],
        [  0, 250,   0],
        [  0, 250,   0]]

In [175]:
# Converter para BGR
bgr_image = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
cv2.imshow("Hue Gradient", bgr_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

- - -